# Data Analysis


In [ ]:
import pandas as pd
import os
import glob
import re

# Define the directory containing the CSV files (modify this path as needed)
input_directory = '/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/clean_data'  # Replace with your directory path
output_file = '/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/samples_features_means.csv'

# Regular expression to extract label, batch, and group from filename
# Assumes filename format like clean_0_190605_6.csv
pattern = r"clean_(\d+)_(\d+)_(\d+)\.csv"

# Ensure the input directory exists
if not os.path.exists(input_directory):
    raise FileNotFoundError(f"The directory {input_directory} does not exist.")

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(input_directory, "*.csv"))

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {input_directory}.")

# Initialize an empty DataFrame to store means and metadata
all_means = pd.DataFrame()

# Process each CSV file
for file in csv_files:
    try:
        # Extract filename from path
        filename = os.path.basename(file)
        
        # Extract label, batch, and group using regex
        match = re.match(pattern, filename)
        if not match:
            print(f"Filename {filename} does not match expected pattern. Skipping.")
            continue
        
        label, batch, group = match.groups()
        
        # Read the CSV file
        df = pd.read_csv(file)
        
        # Compute the mean for each numeric column
        means = df.select_dtypes(include='number').mean()
        
        # Create a DataFrame for the means with the file name as index
        means_df = pd.DataFrame(means).T
        means_df.index = [filename]
        
        # Add metadata columns
        means_df['label'] = int(label)
        means_df['batch'] = int(batch)
        means_df['group'] = int(group)
        
        # Append to the all_means DataFrame
        all_means = pd.concat([all_means, means_df], axis=0)
        
        print(f"Processed {file} successfully.")
        
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")
        continue

# Reorder columns to have label, batch, group first
if not all_means.empty:
    # Get all columns except label, batch, group
    other_columns = [col for col in all_means.columns if col not in ['label', 'batch', 'group']]
    # Reorder columns
    all_means = all_means[['label', 'batch', 'group'] + other_columns]
    
    # Save the means to the output CSV file
    all_means.to_csv(output_file, index=True, index_label='filename')
    print(f"Means and metadata saved to {output_file}")
else:
    print("No data was processed. Output file was not created.")

In [ ]:
df.head()

In [ ]:
df.gene.unique()

### Differential Column Means Analysis (Mutant Vs Wild Type)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu

# Select numeric columns, excluding metadata
required_columns = ['filename', 'label', 'batch', 'group', 'gene']
numeric_columns = [col for col in df.columns if col not in required_columns]

# Separate Mutant (label == 1) and Wild Type (label == 0) groups
mutant_mask = df['label'] == 1
wild_type_mask = df['label'] == 0

mutant_data = df[mutant_mask][numeric_columns]
wild_type_data = df[wild_type_mask][numeric_columns]

# Initialize results DataFrame
results = pd.DataFrame(index=numeric_columns, columns=['t_stat', 't_pvalue', 'mwu_stat', 'mwu_pvalue'])

# Perform t-test and Mann-Whitney U test for each numeric column
for col in numeric_columns:
    # t-test (independent, unequal variances)
    t_stat, t_pval = ttest_ind(mutant_data[col], wild_type_data[col], equal_var=False)
    # Mann-Whitney U test
    mwu_stat, mwu_pval = mannwhitneyu(mutant_data[col], wild_type_data[col], alternative='two-sided')
    # Store results
    results.loc[col, 't_stat'] = t_stat
    results.loc[col, 't_pvalue'] = t_pval
    results.loc[col, 'mwu_stat'] = mwu_stat
    results.loc[col, 'mwu_pvalue'] = mwu_pval

# Save results to CSV
output_file = "/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/statistical_tests_results.csv"
results.to_csv(output_file)

print(f"Statistical test results saved to {output_file}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform

# Load the statistical test results
input_file = "/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/statistical_tests_results.csv"
df = pd.read_csv(input_file, index_col=0)

# Truncate feature names to last 4 characters
feature_labels = [name[-4:] for name in df.index]

# --- Manhattan Plot for T-test ---
plt.figure(figsize=(14, 6))
x = np.arange(len(df.index))
# Calculate -log10(p-values) for t-test
t_logp = -np.log10(df['t_pvalue'])

# Plot t-test -log10(p-values)
plt.scatter(x, t_logp, color='blue', label='T-test (-log10 p-value)', alpha=0.6)

# Add significance threshold (-log10(0.05))
plt.axhline(y=-np.log10(0.05), color='red', linestyle='--', label='Significance (p=0.05)')

# Plot details
plt.xlabel('Features')
plt.ylabel('-log10(P-value)')
plt.title('Manhattan Plot of T-test P-values (Mutant vs Wild Type)')
#plt.xticks(x, feature_labels, rotation=90, ha='right', fontsize=5)
plt.legend()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save t-test Manhattan plot
ttest_plot = '/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/manhattan_plot_ttest.png'
plt.savefig(ttest_plot, dpi=300, bbox_inches='tight')
plt.show()
plt.close()

# --- Manhattan Plot for Mann-Whitney U ---
plt.figure(figsize=(14, 6))
# Calculate -log10(p-values) for Mann-Whitney U
mwu_logp = -np.log10(df['mwu_pvalue'])

# Plot Mann-Whitney U -log10(p-values)
plt.scatter(x, mwu_logp, color='orange', label='Mann-Whitney U (-log10 p-value)', alpha=0.6)

# Add significance threshold (-log10(0.05))
plt.axhline(y=-np.log10(0.05), color='red', linestyle='--', label='Significance (p=0.05)')

# Plot details
plt.xlabel('Features')
plt.ylabel('-log10(P-value)')
plt.title('Manhattan Plot of Mann-Whitney U P-values (Mutant vs Wild Type)')
#plt.xticks(x, feature_labels, rotation=90, ha='right', fontsize=5)
plt.legend()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Save Mann-Whitney U Manhattan plot
mwu_plot = '/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/manhattan_plot_mwu.png'
plt.savefig(mwu_plot, dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print(f"T-test Manhattan plot saved to {ttest_plot}")
print(f"Mann-Whitney U Manhattan plot saved to {mwu_plot}")

In [ ]:
# --- QQ Plot (retained for completeness) ---
def qq_plot(pvalues, title, filename):
    plt.figure(figsize=(8, 8))
    observed = np.sort(pvalues)
    expected = uniform.rvs(size=len(pvalues))
    expected = np.sort(expected)
    
    plt.scatter(-np.log10(expected), -np.log10(observed), color='blue', alpha=0.6)
    plt.plot([0, max(-np.log10(expected))], [0, max(-np.log10(observed))], color='red', linestyle='--')
    
    plt.xlabel('Expected -log10(P-value)')
    plt.ylabel('Observed -log10(P-value)')
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

# Generate QQ plots
qq_plot(df['t_pvalue'], 'QQ Plot of T-test P-values', 
        '/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/qq_plot_ttest.png')
qq_plot(df['mwu_pvalue'], 'QQ Plot of Mann-Whitney U P-values', 
        '/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/qq_plot_mwu.png')

print(f"QQ plot (t-test) saved to /scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/qq_plot_ttest.png")
print(f"QQ plot (Mann-Whitney U) saved to /scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/qq_plot_mwu.png")

## Count the number of features above -log10 

In [2]:
import pandas as pd
import numpy as np

input_file = "/scratch/leuven/359/vsc35907/big_data_feature_extraction/patches_dirs/statistical_tests_results.csv"

# Read the file into a DataFrame (assuming tab-separated values)
df = pd.read_csv(input_file, sep=',')

# Transform p-values using -log10
df['t_log10_pvalue'] = -np.log10(df['t_pvalue'])
df['mwu_log10_pvalue'] = -np.log10(df['mwu_pvalue'])

df.head()

,Unnamed: 0,t_stat,t_pvalue,mwu_stat,mwu_pvalue,t_log10_pvalue,mwu_log10_pvalue
0,feature_0,6.407451,1.048161e-09,14208.0,7.201838e-19,8.979572,18.142557
1,feature_1,4.398852,1.712240e-05,13979.0,1.815278e-17,4.766435,16.741057
2,feature_2,3.350321,9.808892e-04,13751.0,3.954145e-16,3.008380,15.402947
3,feature_3,3.712181,2.707925e-04,13478.0,1.330992e-14,3.567363,13.875825
4,feature_4,1.040826,2.989150e-01,11544.0,4.104928e-06,0.524452,5.386694


In [5]:
# Define thresholds (positive integers for -log10 scale)
thresholds = [1.301, 25, 30, 40]  # Corresponds to p-values of 0.01, 0.001, 0.0001

# Count transformed p-values above each threshold
for threshold in thresholds:
    t_log10_above = (df['t_log10_pvalue'] > threshold).sum()
    mwu_log10_above = (df['mwu_log10_pvalue'] > threshold).sum()
    
    print(f"\nThreshold (-log10 p-value > {threshold})")
    print(f"Number of t-test -log10 p-values > {threshold}: {t_log10_above}")
    print(f"Number of U-test -log10 p-values > {threshold}: {mwu_log10_above}")


Threshold (-log10 p-value > 1.301)
Number of t-test -log10 p-values > 1.301: 879
Number of U-test -log10 p-values > 1.301: 1110

Threshold (-log10 p-value > 25)
Number of t-test -log10 p-values > 25: 51
Number of U-test -log10 p-values > 25: 57

Threshold (-log10 p-value > 30)
Number of t-test -log10 p-values > 30: 42
Number of U-test -log10 p-values > 30: 0

Threshold (-log10 p-value > 40)
Number of t-test -log10 p-values > 40: 9
Number of U-test -log10 p-values > 40: 0


In [6]:
import pandas as pd
import numpy as np

# Transform p-values using -log10
df['t_log10_pvalue'] = -np.log10(df['t_pvalue'])
df['mwu_log10_pvalue'] = -np.log10(df['mwu_pvalue'])

# Define the threshold for significance (p < 0.05 corresponds to -log10(p) > 1.3)
threshold = 1.3

# Identify significant features for each test
t_significant = df[df['t_log10_pvalue'] > threshold]
mwu_significant = df[df['mwu_log10_pvalue'] > threshold]

# Find overlapping significant features (intersection of indices)
overlapping_features = t_significant.index.intersection(mwu_significant.index)
num_overlapping = len(overlapping_features)

# Print results
print(f"Number of significant features (p < 0.05) for t-test: {len(t_significant)}")
print(f"Number of significant features (p < 0.05) for U-test: {len(mwu_significant)}")
print(f"Number of overlapping significant features: {num_overlapping}")

# Optional: List the overlapping features (assuming the first column is the feature name)
if num_overlapping > 0:
    overlapping_feature_names = df.loc[overlapping_features, df.columns[0]].tolist()
    print(f"Overlapping significant features: {overlapping_feature_names}")

Number of significant features (p < 0.05) for t-test: 879
Number of significant features (p < 0.05) for U-test: 1110
Number of overlapping significant features: 873
Overlapping significant features: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_19', 'feature_20', 'feature_21', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_36', 'feature_37', 'feature_38', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_47', 'feature_48', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_55', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69